In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
%matplotlib inline

"""
The following helper functions are copyed from the first assignment 
CarND-LaneLines-P1 for convience and used throughout.
"""
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)   

    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return (line_img, lines)

def weighted_img(img, initial_img, α=1.0, β=1.0, γ=0.0):
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [ ]:
# This method blend/smooths all of the hough lines
# and returns a single left and right lane line
def extract_lane_lines(img, hough_lines):    
    left_count = 0
    right_count = 0
    m_right = 0
    m_left = 0
    b_right = 0
    b_left = 0
    
    for l in hough_lines:
        # each hough_line is defined as a tuple (x1,y1,x2,y2)
        # the slope is defined as ((y2-y1)/(x2-x1))
        # and the intercept is defined as b = y - mx 
        x1, y1, x2, y2 = l[0]
        m = ((y2-y1)/(x2-x1))
        b1 = y1 - m * x1
        b2 = y2 - m * x2
        b = (b1 + b2) / 2.0
        
        # A positive slope is the right line and a negative slope 
        # is the left lane (since image axis are "inverted")
        # Here we add bounds on the slope and intercept to reject outliers
        # and initialize the filtered m and b to the first values
        # that pass the rejection filter        
        if (m > .50 and m < 0.8 and b < (.25 * img.shape[0])):
            m_right += m
            b_right += b
            right_count += 1
        elif (m < -0.60 and m > -0.85 and (b > img.shape[0])):
            m_left += m
            b_left += b
            left_count += 1

    # populate with "reasonable" values if
    # there are no data points from hough_lines
    # that pass the rejection filter
    if (right_count == 0):
        m_right = 0.6
        b_right = 30
        right_count = 1

    if (left_count == 0):
        m_left = -0.6
        b_left = 650
        left_count = 1
            
    # find the average slope and intercept for the
    # left and right lines from all the hough lines 
    # that passed the filter
    m_right /= right_count
    b_right /= right_count
    m_left /= left_count
    b_left /= left_count

    # add some debug information to the frame to show
    # the total number of hough lines, and the number
    # of lines that passed the rejection filter for
    # both the left and the right line
    fontScale = 0.75
    fontType = cv2.FONT_HERSHEY_SIMPLEX
    fontColor = (255, 255, 255)
    cv2.putText(img, 'total hough lines: %d' % len(hough_lines),
                (25, 25), fontType, fontScale, fontColor, 2)

    cv2.putText(img, 'left (%d): m:%.2f b:%.2f' % (left_count, m_left, b_left),
                (25, 50), fontType, fontScale, fontColor, 2)
    
    cv2.putText(img, 'right (%d): m:%.2f b:%.2f' % (right_count, m_right, b_right),
                (25, 75), fontType, fontScale, fontColor, 2)    
    
    def point(m, b, img):
        # Use the equation of a line y = mx + b 
        # and two fixed values of y that correspond
        # to the bottom of the image and 60%
        # through the image to find the x, y points
        y1 = img.shape[0]
        y2 = 0.6 * img.shape[0]
        x1 = (y1 - b) / m
        x2 = (y2 - b) / m
        return [int(x1), int(y1), int(x2), int(y2)]
        
    left_line = [[point(m_left, b_left, img)]]
    right_line = [[point(m_right, b_right, img)]]
    
    return (left_line, right_line)

def pre_process(img):
    vertices = np.array([[(0, img.shape[0]),
                          (int(img.shape[1] * 0.48), int(img.shape[0] * 0.6)),
                          (int(img.shape[1] * 0.52), int(img.shape[0] * 0.6)),
                          (img.shape[1], img.shape[0])]],
                          dtype=np.int32)
    
    grey_image = grayscale(img)
    blurred_image = gaussian_blur(grey_image, 9)
    canny_img = canny(blurred_image, 100, 300)
    trimmed_img = region_of_interest(canny_img, vertices)
    
    # hough parameters are intentionally loose to allow for more 
    # data to pass into the rejection filter implemented in extract_lane_lines
    hough_img, lines = hough_lines(trimmed_img, rho=1, theta=1*math.pi/180.0,
                                   threshold=30, min_line_len=10, max_line_gap=15)
    
    return (hough_img, lines)

def post_process(left_line, right_line, raw_image):
    blank_img = np.zeros((raw_image.shape[0], raw_image.shape[1], 3), dtype=np.uint8)
    draw_lines(blank_img, left_line, color=[0, 255, 0], thickness=7)
    draw_lines(blank_img, right_line, color=[0, 0, 255], thickness=7)
    return weighted_img(blank_img, raw_image)

def save(image, name, directory):
    cv2.imwrite(directory + "/" + i, image)
    
images = os.listdir("test_images/")    
for i in images:
    print("Processing %s" % i)
    raw_img = mpimg.imread('test_images/%s' % i)
    
    # This step takes the raw RGB image and finds
    # the hough image (discarded) and the prominent lines
    hough_img, lines = pre_process(raw_img)
    
    # From the prominent lines, we extract out the "major"
    # left and right line starting from the bottom of the image
    left_line, right_line = extract_lane_lines(raw_img, lines)
    
    # Draw the left and right lane lines onto the original raw image 
    post_img = post_process(left_line, right_line, weighted_img(hough_img, raw_img))
    
    # Save the new image with the drawn lines to disk
    save(post_img, i, "output_images")
    

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # Copy the image argument frame locally
    raw_img = image
    
    # This step takes the raw RGB image and finds
    # the hough image (discarded) and the prominent lines
    hough_img, lines = pre_process(image)
    
    # From the prominent lines, we extract out the "major"
    # left and right line starting from the bottom of the image
    left_line, right_line = extract_lane_lines(raw_img, lines)
    
    # Draw the left and right lane lines onto the original raw image 
    post_img = post_process(left_line, right_line, weighted_img(hough_img, raw_img))
    
    return post_img

In [ ]:
output_video = 'output_images/project_video_output.mp4'
input_video = VideoFileClip('project_video.mp4')
clip = input_video.fl_image(process_image)
%time clip.write_videofile(output_video, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video))